In [1]:
import matplotlib.pyplot as plt
import csv
import heartpy as hp
import numpy as np
from sklearn.decomposition import FastICA
import scipy
from scipy import signal
import statistics
import statsmodels.api as sm
from scipy.stats.kde import gaussian_kde

In [2]:
x_ECG = []
x_radar = []
ECG = []
time_ECG = []
time_radar= []
I = []
Q = []
time_difference=1.34


with open('C://Users//gy299//20230517-141844-ecg.csv','r') as csvfile:
    lines = csv.reader(csvfile, delimiter=',')
    for row in lines:
        ECG.append((int(row[0])))

with open('C://Users//gy299//20230517-141844-radar.csv','r') as csvfile:
    lines = csv.reader(csvfile, delimiter=',')
    for row in lines:
        I.append((float(row[0])))
        Q.append(float(row[1]))

length=len(ECG)
for i in range(1,length+1):
    x_ECG.append(i)
    time_ECG.append(0.002*i)

length=len(I)
for i in range(1,length+1):
    x_radar.append(i)
    time_radar.append(0.0005*i)

flag=True
for i in range(len(time_radar)):
    true_time=time_radar[i]-time_difference
    if true_time>=0 and flag==True:
        index=i
        flag=False

time_radar=time_radar[index:]
time_radar=[x-time_difference for x in time_radar]
I=I[index:]
Q=Q[index:]

#DC component removal
I=scipy.signal.detrend(I, axis=-1, type='constant', bp=0, overwrite_data=False)
Q=scipy.signal.detrend(Q, axis=-1, type='constant', bp=0, overwrite_data=False)

In [ ]:
#raw ECG signal plot
plt.figure(figsize=(12,4))
plt.plot(time_ECG,ECG, color = 'g')
plt.xlabel('time(sec)')
plt.ylabel('amplitude')
plt.title('Raw ECG signal')
plt.show()
plt.figure(figsize=(12,4))
plt.plot(time_ECG,ECG, color = 'g')
plt.xlabel('time(sec)')
plt.ylabel('amplitude')
plt.xlim(0,10)
plt.title('A closer look at the raw ECG signal')
plt.show()

In [ ]:
sample_rate=500
#detrend the ECG signal to remove the baseline drift
detrended_ECG = signal.detrend(ECG)
#apply bandpass filtering on the detrended ECG signal
filtered_ECG = hp.filter_signal(detrended_ECG, cutoff = [2,40], sample_rate = sample_rate,order = 6, filtertype= 'bandpass')

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(time_ECG,filtered_ECG, color = 'g')
plt.xlabel('time(sec)')
plt.ylabel('amplitude')
plt.title('ECG signal after detrending and filtering')
plt.show()
plt.figure(figsize=(12,4))
plt.plot(time_ECG,filtered_ECG, color = 'g')
plt.xlabel('time(sec)')
plt.ylabel('amplitude')
plt.xlim(0,10)
plt.title('A closer look at the ECG signal after detrending and filtering')
plt.show()

In [ ]:
wd, m = hp.process(filtered_ECG, sample_rate, calc_freq=False,freq_method='fft')

In [ ]:
plt.figure(figsize=(12,4))
plot_object = hp.plotter(wd, m, show=False)
#plot_object.savefig('result.jpg') #saves the plot as JPEG image.
plot_object.show() #displays plot
#display measures computed
for measure in m.keys():
    print('%s: %f' %(measure, m[measure]))

In [ ]:
array=np.array(wd['peaklist'])/sample_rate

In [ ]:
#raw radar signal plot
plt.figure(figsize=(12,4))
plt.plot(time_radar,I, color = 'g',label='radar_I')
plt.plot(time_radar, Q, color = 'b', label='radar_Q')
plt.xlabel('time(sec)')
plt.ylabel('amplitude')
plt.vlines(x = array, ymin=-100,ymax=500,
           colors = 'r',linestyles='dotted',
           label = 'R_peak')
plt.ylim(0,1)
plt.legend()
plt.title('raw radar signal')
plt.show()

In [ ]:
#apply bandpass filtering on the radar I/Q channels
#cutoff= [15,25]
#cutoff=[0.7,3]
cutoff=[25,40]
sample_rate=2000
filtered_I = hp.filter_signal(I, cutoff = cutoff, sample_rate = sample_rate,order = 4, filtertype= 'bandpass')
filtered_Q = hp.filter_signal(Q, cutoff = cutoff, sample_rate = sample_rate,order = 4, filtertype= 'bandpass')

In [ ]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid')

In [ ]:
complex_signal=np.zeros(len(I), dtype='complex')
complex_signal.imag=filtered_I
complex_signal.real=filtered_Q
IQ=np.abs(complex_signal)

#IQ_smooth = signal.savgol_filter(IQ, window_length=667, polyorder=10, mode="mirror")
#IQ_ma=moving_average(IQ_smooth,500)
IQ_f=hp.filter_signal(IQ, cutoff = [0.7,3], sample_rate = sample_rate,order = 2, filtertype= 'bandpass')
IQ_smooth = signal.savgol_filter(IQ_f, window_length=667, polyorder=10, mode="mirror")
IQ_ma=moving_average(IQ_smooth,500)

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(time_radar[499:],IQ_ma, color = 'g',label='IQ after applying moving average filter and smooth filter')
plt.vlines(x = array, ymin=-2*((10)**(-19)),ymax=2*((10)**(-19)),
           colors = 'r',linestyles='dotted',
           label = 'R_peak')
plt.ylim(-2*((10)**(-19)),2*((10)**(-19)))
plt.xlabel('time')
plt.ylabel('magnitude')
plt.title('IQ_ma')
plt.show()

In [ ]:
wd, m = hp.process(IQ_ma, sample_rate, calc_freq=False,freq_method='fft',bpmmin=40, bpmmax=180,reject_segmentwise=False)

In [ ]:
#good quality segments: 110250 10-50; 113414 10-39;
start_time=0
end_time=30
sample_rate=2000
wd, m = hp.process(IQ_ma[sample_rate*start_time:sample_rate*end_time], sample_rate, calc_freq=False,freq_method='fft', bpmmin=40, bpmmax=180)

In [ ]:
new_array=[]
for i in range(len(array)):
    if array[i]>=start_time and array[i]<=end_time:
        new_array.append(array[i]-start_time)
plt.figure(figsize=(12,4))
plot_object = hp.plotter(wd, m, show=False)
plt.vlines(x = array, ymin=1*((10)**(-19)),ymax=5*((10)**(-19)),
           colors = 'r',linestyles='dotted',
           label = 'R_peak')
#plt.ylim(1*((10)**(-19)),5*((10)**(-19)))
plt.legend()
#plot_object.savefig('result.jpg') #saves the plot as JPEG image.
plot_object.show() #displays plot


#display measures computed
for measure in m.keys():
    print('%s: %f' %(measure, m[measure]))

In [ ]:
#find peaks function (prominence-distance-based method)
from scipy.signal import find_peaks
sample_rate=2000
IBI_list_radar=[]
HR_list_radar=[]
data = IQ_ma
print(abs(data.mean()*1000))
peaks, properties = find_peaks(data, distance=667,prominence=(0.01*abs(data.mean()*1000), None),width=None, height=None)
for i in range(1, len(peaks)):
    IBI_list_radar.append((peaks[i]-peaks[i-1])/2000)

HR_list_radar=[60/x for x in IBI_list_radar]
print('Pulse Number: {}'.format(len(peaks)))
print('The average IBI is {} ms'.format(round(statistics.mean(IBI_list_radar)*1000,2)))
print('HR: {} bpm'.format(round(statistics.mean(HR_list_radar),2)))

plt.figure(figsize=(12,4))
plt.plot(time_radar[499:],data)
peak_index = [time_radar[499:][x] for x in peaks]
plt.plot(peak_index, data[peaks], "o",color = 'g')
plt.vlines(x = array, ymin=-2*((10)**(-19)),ymax=2*((10)**(-19)),
           colors = 'r',linestyles='dotted',
           label = 'R_peak')
plt.ylim(-2*((10)**(-19)),2*((10)**(-19)))
plt.ylim()
plt.show()

In [ ]:
new_array =[]
new_peak_index = []
IBI_list_ECG = []
IBI_list_radar = []
time_range =[0,30]
for index in array:
    if index>time_range[0] and index<time_range[1]:
        new_array.append(index)
for index in peak_index:
    if index>time_range[0] and index<time_range[1]:
        new_peak_index.append(index)
for i in range(1, len(new_array)):
    IBI_list_ECG.append((new_array[i]-new_array[i-1]))
for i in range(1, len(new_peak_index)):
    IBI_list_radar.append((new_peak_index[i]-new_peak_index[i-1]))

HR_ECG = [60/x for x in IBI_list_ECG]
HR_radar = [60/x for x in IBI_list_radar]

print(len(IBI_list_radar))
print(len(IBI_list_ECG))
#print(IBI_list_rPPG)
#print(IBI_list_ECG)
print('ECG: {} bpm'.format(np.mean(HR_ECG)))
print('rPPG: {} bpm'.format(np.mean(HR_radar)))

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(time_radar[499:],data,label='rPPG')
peak_index = [time_radar[499:][x] for x in peaks]
plt.plot(peak_index, data[peaks], "o",color='g')
plt.vlines(x = array, ymin=-2*((10)**(-19)),ymax=2*((10)**(-19)),
           colors = 'r',linestyles='dotted',
           label = 'R_peak')
plt.ylim(-2*((10)**(-19)),2*((10)**(-19)))
plt.xlim(time_range[0],time_range[1])
plt.legend()
plt.show()

In [ ]:
if len(HR_ECG)==len(HR_radar):
    print('synchronised')
    plt.figure(figsize=(12,4))
    plt.plot(HR_ECG,color='g',label='HR_ECG')
    plt.plot(HR_radar,color='b',label='HR_radar')
    plt.ylabel('bpm')
    plt.xlabel('nth HR estimation')
    plt.legend()
    plt.title('Comparison between the HR estimations from ECG and Radar')
    plt.show()

    MAE=0
    AE=0
    for i in range(len(HR_ECG)):
        #print(abs(HR_ECG[i]-HR_radar[i]))
        MAE+=abs(HR_ECG[i]-HR_radar[i])
        AE+=(abs(HR_ECG[i]-HR_radar[i])/HR_ECG[i])*100
    
    MAE = MAE/len(HR_ECG)
    AE = AE/len(HR_ECG)
    print('The Mean Absolute Error is: {} bpm\nThe Average Error is: {} %'.format(MAE,AE))
else:
    print('desynchronised!')

In [ ]:
import statsmodels.api as sm
ecg=[]
radar=[]

In [ ]:
ecg=ecg+HR_ECG
radar=radar+HR_radar

In [ ]:
f, ax = plt.subplots(1, figsize = (8,5))
sm.graphics.mean_diff_plot(np.array(ecg), np.array(radar), ax = ax)
plt.xlabel('Mean of HR_ECG and HR_radar')
plt.ylabel('Mean difference (bmp)')
plt.title('bland-altman plot')
plt.show()

In [ ]:
IBI_list_ECG = [(60/x)*1000 for x in ecg]
IBI_list_radar = [(60/x)*1000 for x in radar]

In [ ]:
IBI_list_ECG = [x*1000 for x in IBI_list_ECG]
IBI_list_radar =[x*1000 for x in IBI_list_radar]

In [ ]:
plt.figure(figsize=(12,5))
plt.hist((IBI_list_ECG, IBI_list_radar),bins=30, histtype='bar', rwidth=0.85, label=('IBI_ECG','IBI_radar'),color=('g','b'),density=True)

mn, mx = plt.xlim()
plt.xlim(mn, mx)
kde_xs = np.linspace(mn, mx, 500)
kde = gaussian_kde(IBI_list_ECG)
plt.plot(kde_xs, kde.pdf(kde_xs), label="PDF_ECG",color='g')

kde_xs = np.linspace(mn, mx, 500)
kde = gaussian_kde(IBI_list_radar)
plt.plot(kde_xs, kde.pdf(kde_xs), label="PDF_radar",color='b')

plt.xlabel('interbeat interval (IBI) [ms]')
plt.ylabel('Probability density [%]')
plt.title('Histogram of IBI distribution of ECG and radar')
plt.legend()
plt.show() 

plt.figure(figsize=(12,5))
plt.hist((IBI_list_ECG, IBI_list_radar),bins=20, histtype='bar', rwidth=0.85, label=('IBI_ECG','IBI_radar'),color=('g','b'), density=True)
mn, mx = plt.xlim()
plt.xlim(mn, mx)
kde_xs = np.linspace(mn, mx, 500)
kde = gaussian_kde(IBI_list_ECG)
plt.plot(kde_xs, kde.pdf(kde_xs), label="PDF_ECG",color='g')

kde_xs = np.linspace(mn, mx, 500)
kde = gaussian_kde(IBI_list_radar)
plt.plot(kde_xs, kde.pdf(kde_xs), label="PDF_radar",color='b')
plt.xlabel('interbeat interval (IBI) [ms]')
plt.ylabel('Probability density [%]')
plt.title('Histogram of IBI distribution of ECG and radar')
plt.legend()
plt.show() 


plt.figure(figsize=(12,5))
plt.hist((IBI_list_ECG, IBI_list_radar),bins=10, histtype='bar', rwidth=0.85, label=('IBI_ECG','IBI_radar'),color=('g','b'), density=True)
mn, mx = plt.xlim()
plt.xlim(mn, mx)
kde_xs = np.linspace(mn, mx, 500)
kde = gaussian_kde(IBI_list_ECG)
plt.plot(kde_xs, kde.pdf(kde_xs), label="PDF_ECG",color='g')

kde_xs = np.linspace(mn, mx, 500)
kde = gaussian_kde(IBI_list_radar)
plt.plot(kde_xs, kde.pdf(kde_xs), label="PDF_rPPG",color='b')
plt.xlabel('interbeat interval (IBI) [ms]')
plt.ylabel('Probability density [%]')
plt.title('Histogram of IBI distribution of ECG and radar')
plt.legend()
plt.show() 

In [ ]:
print(len(IBI_list_ECG))